# Module 1B
### Generating Ome.tiff files containing Regions of Interest (ROI) for individual cells

### This module takes as input a folder of images and a corresponding folder of indexed images ("masks") in which the value of each pixel is the index (number) of the cell that contains that pixel (and pixels outside cells are zero).

### The files in the folders must be in corresponding lexigraphical order.

In [1]:
# Code Cell 1: run this cell to define the functions used in the later cells.

from aicsimageio import AICSImage, writers, vendor
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon
from pathlib import Path
from typing import Union, Sequence
import numpy as np
#from argparse import ArgumentParser
import re
import os

INTEGER_PATTERN = re.compile(r'(\d+)')
FILENAMES_TO_IGNORE = frozenset({'.DS_Store'})

module_name = 'Module5'
topdir = "/home/murphylab/cellorganizer/local/results"
outputdir = topdir + "/" + module_name
if not os.path.exists(topdir):
    os.makedirs(topdir)
os.chdir(topdir)
os.system("ls")
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
os.chdir(outputdir)
os.system("ls")


def try_parse_int(value: str) -> Union[int, str]:

    if value.isdigit():
        return int(value)
    return value


def alphanum_sort_key(path: Path) -> Sequence[Union[int, str]]:
    """
    Produces a sort key for file names, alternating strings and integers.
    Always [string, (integer, string)+] in quasi-regex notation.
    >>> alphanum_sort_key(Path('s1 1 t.tiff'))
    ['s', 1, ' ', 1, ' t.tiff']
    >>> alphanum_sort_key(Path('0_4_reg001'))
    ['', 0, '_', 4, '_reg', 1, '']
    """
    return [try_parse_int(c) for c in INTEGER_PATTERN.split(path.name)]


def get_paths(img_dir: Path) -> Sequence[Path]:
    if img_dir.is_dir():
        img_files = [c for c in img_dir.iterdir() if c.name not in FILENAMES_TO_IGNORE]
    else:
        # assume it's a pattern, like Path('some/dir/*.tiff')
        # don't need to filter filenames, because the user passed a
        # glob pattern of exactly what is wanted
        img_files = list(img_dir.parent.glob(img_dir.name))

    return sorted(img_files, key=alphanum_sort_key)

def create_roi_polygons(
        mask: np.ndarray,
        omeXml,
) :
    """
    Given a NumPy ndarray of image data and an aicsimageio.vendor.omexml.OMEXML object
    containing basic OME-XML, create strings representing the polygon shapes of segmented cells
    and add these to the "ROI" element of the OME-XML. Return the OME-XML with the ROI element
    populated.
    """

    #3D mask -> Should put in channels?
    mask3D = mask.data[0, 0, 0, :, :, :]
    omeXmlRoot = ET.fromstring( omeXml.to_xml() )

    for i in range(1, np.max(mask.data) + 1):
        roiRef = ET.SubElement(
            omeXmlRoot[0],
            "ROIRef",
            attrib = {
                "ID" : "ROI:" + str(i - 1)
            }

        )
        roiElement = ET.SubElement(
            omeXmlRoot,
            "ROI",
            attrib = {
                "xmlns" : "http://www.openmicroscopy.org/Schemas/ROI/2015-06",
                "ID" : "ROI:" + str(i - 1)
            },
        )
        # roiDesc = ET.SubElement( roiElement, "Description" )
        # roiDesc.text = "Polygon points of mask segmentation"

        roiUnion = ET.SubElement( roiElement, "Union")

        #start of finding regions
        roiShape = np.where( mask3D == i )
        count = 0
        roiShapeTuples = list(zip(roiShape[2], roiShape[1]))
        polygon = Polygon(roiShapeTuples)
        coords = polygon.exterior.coords
        coordStrings = []

        for j in range(0, len(roiShape[0]) - 1):

            if roiShape[0][j] == roiShape[0][j+1]:
                coordPairString = ",".join( [ str( int( c ) ) for c in coords[j] ] )
                coordStrings.append( coordPairString )

                # TOFIX: Not ideal but for the sake of time
                if int(j) == len(roiShape[0]) - 2:
                    j += 1
                    coordPairString = ",".join([str(int(c)) for c in coords[j]])
                    coordStrings.append(coordPairString)

                    allCoordsString = " ".join(coordStrings)

                    # Create Polygon with coordinates in.
                    roiPolygon = ET.SubElement(
                        roiUnion,
                        "Polygon",
                        attrib={
                            "ID": "Shape:" + str(count),
                            "Points": allCoordsString,
                            "TheZ": str(roiShape[0][j]),
                        },
                    )

            else:
                coordPairString = ",".join( [ str( int( c ) ) for c in coords[j] ] )
                coordStrings.append( coordPairString )

                allCoordsString = " ".join(coordStrings)

                # Create Polygon with coordinates in.
                roiPolygon = ET.SubElement(
                    roiUnion,
                    "Polygon",
                    attrib={
                        "ID": "Shape:" + str(count),
                        "Points": allCoordsString,
                        "TheZ": str(roiShape[0][j]),
                    },
                )

                coordStrings = []
                count += 1

    omeXmlWithROIs = vendor.omexml.OMEXML( xml = ET.tostring( omeXmlRoot ) )

    return omeXmlWithROIs

def write_2_ometiff(newfilename, img, omeXml):

    #wrt = writers.OmeTiffWriter(newfilename, overwrite_file=True)
    wrt = writers.OmeTiffWriter(newfilename)
    wrt.save(img.data[0,0,:,:,:,:], ome_xml = omeXml, dimension_order='CZYX')


def initOmeXml(img):

    omeXml = vendor.omexml.OMEXML()
    omeXml.image().Pixels.set_SizeT(img.size_t)
    omeXml.image().Pixels.set_SizeC(img.size_c)
    omeXml.image().Pixels.set_SizeZ(img.size_z)
    omeXml.image().Pixels.set_SizeY(img.size_y)
    omeXml.image().Pixels.set_SizeX(img.size_x)
    omeXml.image().Pixels.set_PixelType(str(img.data.dtype))
    omeXml.image().Pixels.set_DimensionOrder("XYCZT")

    return omeXml

### Input Directories: 

Modify this cell to contain the folders of your own images and masks. 
Give the absolute path if it resides outside of this working directory (where the jupyter notebook resides). If the two directories lie within the juptyer notebook you can use relative pathing. 

In [2]:
# Code Cell 2

# this image 
image_directory_path = '/home/murphylab/cellorganizer/local/images/CODEX/intensities'
# the mask in this folder only contains the first 50 cells (in order to make subsequent model building faster)
mask_directory_path = '/home/murphylab/cellorganizer/local/images/CODEX/masks'
# change to this folder to use all cells in the images (about 2300)
#mask_directory_path = '/home/murphylab/cellorganizer/local/images/CODEX/fullmask'

### Write out OMETIFFs

Run the cell below to create OMETIFFs of your own images that will be readable by Cellorganizer!

Note that the input images must be 2D or 3D multicolor images for a single region and timepoint, that is, the CXYZ dimensions can be greater than 1 but the R and T dimensions must be 1. 

If your images have more than one region or timepoint, you can modify the code as illustrated below to specify which region or timepoint to select.

In [3]:
# Code Cell 3

img_files = get_paths(Path(image_directory_path))
mask_files = get_paths(Path(mask_directory_path))

# matching images and masks must be in lexical order
img_files.sort()
mask_files.sort()

for i in range(0,len(img_files)):

    mask = AICSImage(mask_files[i])
    img = AICSImage(img_files[i])

    assert(img.shape[0] or img.shape[1] is 1), \
        "Image contains multiple regions or multiple timepoints."

    # reg = 1 # change this as needed
    # tpoint = 1 # change this as needed
    # img = img[reg,tpoint,:,:,:,:]
    omeXml = initOmeXml(img)

    omeXml = create_roi_polygons(mask, omeXml)

    headtail = os.path.split(img_files[i])
    # write the file to the output directory using the same name as the input files
    write_2_ometiff(outputdir + os.path.sep + headtail[1], img, omeXml)